In [2]:
import glob 
import os
from PIL import Image
import matplotlib.pyplot as plt
from PIL import ImageFile
import numpy as np
import json
%matplotlib inline
from tqdm import tqdm

from random import randrange

In [3]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [4]:
def padding(img_path):
    img = Image.open(img_path)
    img = img.convert('RGB')
    w, h = img.size
    img = np.array(img)
    
    margin = [np.abs(w - h) // 2 , np.abs(w - h) // 2]
    if w > h:
        margin_list = [margin, [0,0]]
    else:
        margin_list = [[0,0], margin]
        
    if len(img.shape) == 3:
        margin_list.append([0,0])
        
    output = np.pad(img, margin_list, mode='constant')
    img = Image.fromarray(output)
    # 이미지 크기
    img = img.resize((224, 224))
    #img = np.array(img)
    return img

In [5]:
def status_to_int(status):
    if status == 'NOR': #정상
        int_status = 0
    else:
        int_status = 1
    return int_status

In [6]:
def json_name_to_img_path(json_name):
    img_name = json_name.split('.')[0] +'.jpg'
    img_path = img_dir_path + '/'+ img_name
    return img_path

In [7]:
di_class = ['Mu01','Mu02','Mu03'] #,'Mu04','Mu05','Ab06','Ab07','Ab08','Ab09','Ab10']

di =  'Mu02' 
train_val = 't'

if train_val == 't':
    img_dir_path = 'C:/Users/seonh/Downloads/062.반려동물 질병 진단을 위한 영상 데이터 (근골격계)/01.데이터/Training/원천데이터/' + di
    json_dir_path = 'C:/Users/seonh/Downloads/062.반려동물 질병 진단을 위한 영상 데이터 (근골격계)/01.데이터/Training/라벨링데이터/TL2_DOG'
    
else:
    img_dir_path = 'C:/Users/seonh/Downloads/062.반려동물 질병 진단을 위한 영상 데이터 (근골격계)/01.데이터/Validation/원천데이터/sep/' + di
    json_dir_path = 'C:/Users/seonh/Downloads/062.반려동물 질병 진단을 위한 영상 데이터 (근골격계)/01.데이터/Validation/라벨링데이터/VL2_DOG'
    
json_name_list = os.listdir(json_dir_path)  # 다 영어로..?

In [285]:
print(len(json_name_list))

print(json_name_list[:10])

113725
['D_62_19980513_IF_0028_NOR_Mu01_20211216_0165.json', 'D_62_19980513_IF_0028_NOR_Mu01_20211216_0167.json', 'D_62_19980513_IF_0028_NOR_Mu01_20211216_0168.json', 'D_62_19980513_IF_0028_NOR_Mu01_20211216_0169.json', 'D_62_19980615_IF_0028_NOR_Mu07_20211216_0171.json', 'D_62_19980623_IM_0028_NOR_Mu01_20211216_0173.json', 'D_62_19980623_IM_0028_NOR_Mu01_20211216_0174.json', 'D_62_19980623_IM_0028_NOR_Mu01_20211216_0175.json', 'D_62_19980623_IM_0028_NOR_Mu01_20211216_0176.json', 'D_62_19980623_IM_0028_NOR_Mu01_20211216_0177.json']


In [12]:
err = 0
norm_num = 0
abb = 0
numb = -1
x_data, y_data = [], []
resize = 224
norm_idx = []

rot = [-5, 5]
trans = [[resize/10, 0], [0,(resize/10)] ] #, [0,-(resize/10)], [-(resize/10), 0]]

V, L, A, M = [], [], [], []
ab_v, ab_l, ab_a, ab_m = 0, 0, 0, 0

pos_spec = ['_L']

for json_name  in tqdm(json_name_list):
    if json_name.split('_')[6] == di:
        json_path = json_dir_path + '/'+ json_name
        try:
            with open(json_path, 'r', encoding='utf-8') as f:
                
                
                data = json.load(f)
                status = data["metadata"]['Disease']
                pose = data["metadata"]['Position']
                int_status = status_to_int(status)
                
                
                if pose == "VD":
                    continue
                    numb +=1
                    V.append(numb)
                    
                    
                    
                elif pose == "Lateral":
                    #continue
                    numb +=1
                    L.append(numb)
                    
                    
                    
                elif pose == "AP":
                    continue
                    numb +=1
                    A.append(numb)
                    
                    
                    
                    
                else:
                    continue
                    numb +=1
                    
                    M.append(numb)
                    #continue
                    
                    
                    
                  
                #y_data.append(numb)
                y_data.append(int_status)
            
                    
                img_path = json_name_to_img_path(json_name)
                                
            img = padding(img_path)    
            img_f = np.array(img)
            

            
            x_data.append(img_f)


            
            if y_data[-1] == 0  :
                norm_num +=1
                
                norm_idx.append(numb)
                if numb >=len(x_data):
                    print(numb, 'x 길이', len(x_data))
                    
                if train_val == 't':
                    for angle in rot:   #test_img = test_img.rotate(-5)
                        ss = randrange(len(x_data)+1)
                        y_data.insert(ss, y_data[-1])

                        rot_img = img.rotate(angle)
                        np_img = np.array(rot_img)
                        x_data.insert(ss, np_img)

                        norm_idx.append(ss)

                    for k in trans:
                        ss = randrange(len(x_data)+1)
                        y_data.insert(ss, y_data[-1])

                        tran_img = img.rotate(0, translate = k)
                        np_img = np.array(tran_img)
                        x_data.insert(ss, np_img)

                        norm_idx.append(ss)

                
            else:
                abb +=1
                
                if pose == "VD":
                    ab_v+= 1
                    
                elif pose == "Lateral":
                    ab_l +=1
                elif pose == "AP":
                    ab_a +=1
                else:
                    ab_m +=1
                
                if train_val == 't':
                    for angle in rot:   #test_img = test_img.rotate(-5)
                        ss = randrange(len(x_data)+1)
                        y_data.insert(ss, y_data[-1])

                        rot_img = img.rotate(angle)
                        np_img = np.array(rot_img)
                        x_data.insert(ss, np_img)
                        #train_set.append(value)

                    for k in trans:
                        ss = randrange(len(x_data)+1)
                        y_data.insert(ss, y_data[-1])

                        tran_img = img.rotate(0, translate = k)
                        np_img = np.array(tran_img)
                        x_data.insert(ss, np_img)  

                
                
        except FileNotFoundError:
            err +=1
            print(json_path)

print('ABS:', abb)
print('Normal', norm_num)
print('err:', err)

  6%|▋         | 7110/113725 [00:17<04:25, 400.84it/s] 


KeyboardInterrupt: 

In [97]:
print(len(V))
print(len(L))
print(len(A))
print(len(M))
print(len(norm_idx)/5,'aug 후 ',len(norm_idx))
print( len(M)-len(norm_idx)/5 )
print(len(x_data), len(y_data))
print(numb)

0
10
4
2225
426.2 aug 후  2131
1798.8
2225 2225
2238


In [331]:
print(len(V))
print(len(L))
print(len(A))
print(len(M))
x_data_V, y_data_V, x_data_L, y_data_L, x_data_A, y_data_A, x_data_M, y_data_M = [], [], [], [], [], [], [], [] 

#for i in range(numb+1):
for i in V:
    x_data_V.append(x_data[i])
    y_data_V.append(y_data[i])

for i in L:
    x_data_L.append(x_data[i])
    y_data_L.append(y_data[i])

for i in A:
    x_data_A.append(x_data[i])
    y_data_A.append(y_data[i])

for i in M:
    x_data_M.append(x_data[i])
    y_data_M.append(y_data[i])


28
5
8034
12248


In [81]:
print('전체 aug ', len( x_data ))
print('전체 normal', len( norm_idx ))


print(abb*7)

print('VD 전체, NOR, AB 수', len(V), len(V)-ab_v ,ab_v)
print('Lateral 전체, NOR, AB 수',len(L), len(L)-ab_l ,ab_l)
print('AP 전체, NOR, AB 수',len(A), len(A)-ab_a ,ab_a)
print('ML 전체, NOR, AB 수',len(M), len(M)-ab_m ,ab_m)


print('vale_numb', val_numb*4)
val_numb = 2483
num_drop = len( x_data )-val_numb*4
print(num_drop)


전체 aug  41515
전체 normal 36395
7168
VD 전체, NOR, AB 수 0 0 0
Lateral 전체, NOR, AB 수 0 0 0
AP 전체, NOR, AB 수 0 0 0
ML 전체, NOR, AB 수 8303 7279 1024


NameError: name 'val_numb' is not defined

In [160]:
print('ABN aug 수',len(x_data)- len(norm_idx))
print(len(x_data), len(y_data))
#print(len())
print(len(norm_idx))

num_drop = 2*len(norm_idx) -len(x_data)  #len(norm_idx) - len(x_data)-(len(norm_idx) )
print(num_drop)
print(len(x_data), len(norm_idx), len(x_data)-len(norm_idx))

ABN aug 수 1172
2471 2471
1299
127
2471 1299 1172


In [182]:
print(len(x_data))
print(len(norm_idx))
print(len(remo))
print(remo[-9:])
print(sorted(norm_idx)[-1])

96
1477
1429
[1516, 1517, 1518, 1519, 1520, 1521, 1522, 1523, 1524]
1524


In [181]:
import random

num_drop = 2*len(norm_idx) -len(x_data)  #len(norm_idx) - len(x_data)-(len(norm_idx) )

#remo = sorted(random.sample(drop_t,num_drop))
##remo = sorted(random.sample(list(range(len(x_data))),num_drop))
remo = sorted(random.sample(norm_idx,num_drop))

# print(remo[-i-1])
# print(remo[-1])
# print(remo[-9:])
#print(x_data)


for i in range(len(remo)):
    del x_data[remo[-i-1]]
    del y_data[remo[-i-1]]



In [47]:
print(type(x_data))
import pickle
with open('t_Mu03_x_data.pkl', 'wb') as f:
    pickle.dump(x_data,f)

<class 'list'>


In [183]:
x_data = np.array(x_data)
y_data = np.array(y_data)

In [175]:
x_data.shape

(1525, 224, 224, 3)

In [204]:
y_data.shape

(5618,)

In [16]:
print(y_data)

[0 0 0 ... 0 0 0]


In [184]:

for i in pos_spec: 
    print(i)
    if train_val == 't':

        a_path = "C:/Users/seonh/Downloads/062.반려동물 질병 진단을 위한 영상 데이터 (근골격계)/01.데이터/Training/augment"

        np.save(a_path +'/'+di+ i +'_x_data_tr_ch_1000.npy', x_data)
        np.save(a_path +'/'+di+  i +'_y_data_tr_ch_1000.npy', y_data)

    else:

        dir_path = "C:/Users/seonh/Downloads/062.반려동물 질병 진단을 위한 영상 데이터 (근골격계)/01.데이터/Validation/numpy_save"

        np.save(dir_path +'/'+di+i+'_x_data_val_1000.npy', x_data)
        np.save(dir_path +'/'+di+i+'_y_data_val_1000.npy', y_data)


_L


In [11]:
#di_class = {'Mu01': ['_A', '_M'],  'Mu02': ['_A', '_M'],  'Mu04': ['_A'], 'Mu03': ['_V', '_L'] }
di_class = {   'Mu03': ['_V', '_L'] }
train_val = ['t', 'ss']

import random

for t_v in train_val:
  for state in di_class:
    di = state
    train_val = t_v
    
    
    if train_val == 't':
        img_dir_path = 'C:/Users/seonh/Downloads/062.반려동물 질병 진단을 위한 영상 데이터 (근골격계)/01.데이터/Training/원천데이터/' + di
        json_dir_path = 'C:/Users/seonh/Downloads/062.반려동물 질병 진단을 위한 영상 데이터 (근골격계)/01.데이터/Training/라벨링데이터/TL2_DOG'

    else:
        img_dir_path = 'C:/Users/seonh/Downloads/062.반려동물 질병 진단을 위한 영상 데이터 (근골격계)/01.데이터/Validation/원천데이터/sep/' + di
        json_dir_path = 'C:/Users/seonh/Downloads/062.반려동물 질병 진단을 위한 영상 데이터 (근골격계)/01.데이터/Validation/라벨링데이터/VL2_DOG'
    json_name_list = os.listdir(json_dir_path)  # 다 영어로..?
    

    for posis in di_class[state]:

        err = 0
        norm_num = 0
        abb = 0
        numb = -1
        x_data, y_data = [], []
        resize = 224
        norm_idx = []

        rot = [-5, 5]
        trans = [[resize/10, 0], [0,(resize/10)] ] #, [0,-(resize/10)], [-(resize/10), 0]]

        V, L, A, M = [], [], [], []
        ab_v, ab_l, ab_a, ab_m = 0, 0, 0, 0

        pos_spec = posis

        for json_name  in tqdm(json_name_list):
            if json_name.split('_')[6] == di:
                json_path = json_dir_path + '/'+ json_name
                try:
                    with open(json_path, 'r', encoding='utf-8') as f:


                        data = json.load(f)
                        status = data["metadata"]['Disease']
                        pose = data["metadata"]['Position']
                        int_status = status_to_int(status)


                        if pose == "VD":
                            if pos_spec == '_V':
                                numb +=1
                                V.append(numb)
                            else:
                                continue

                        elif pose == "Lateral":
                            if pos_spec == '_L':
                                numb +=1
                                L.append(numb)
                            else:
                                continue

                        elif pose == "AP":
                            if pos_spec == '_A':
                                numb +=1
                                A.append(numb)
                            else:
                                continue                    

                        else:
                            if pos_spec == '_M':
                                numb +=1
                                M.append(numb)
                            else:
                                continue


                        #y_data.append(numb)
                        y_data.append(int_status)


                        img_path = json_name_to_img_path(json_name)

                    img = padding(img_path)    
                    img_f = np.array(img)


                    x_data.append(img_f)

                    if y_data[-1] == 0  :
                        norm_num +=1

                        norm_idx.append(numb)
                        if numb >=len(x_data):
                            print(numb, 'x 길이', len(x_data))

                        if train_val == 't':
                            for angle in rot:   #test_img = test_img.rotate(-5)
                                ss = randrange(len(x_data)+1)
                                y_data.insert(ss, y_data[-1])

                                rot_img = img.rotate(angle)
                                np_img = np.array(rot_img)
                                x_data.insert(ss, np_img)

                                norm_idx.append(ss)

                            for k in trans:
                                ss = randrange(len(x_data)+1)
                                y_data.insert(ss, y_data[-1])

                                tran_img = img.rotate(0, translate = k)
                                np_img = np.array(tran_img)
                                x_data.insert(ss, np_img)

                                norm_idx.append(ss)

                    else:
                        abb +=1

                        if pose == "VD":
                            ab_v+= 1

                        elif pose == "Lateral":
                            ab_l +=1
                        elif pose == "AP":
                            ab_a +=1
                        else:
                            ab_m +=1

                        if train_val == 't':
                            for angle in rot:   #test_img = test_img.rotate(-5)
                                ss = randrange(len(x_data)+1)
                                y_data.insert(ss, y_data[-1])

                                rot_img = img.rotate(angle)
                                np_img = np.array(rot_img)
                                x_data.insert(ss, np_img)
                                #train_set.append(value)

                            for k in trans:
                                ss = randrange(len(x_data)+1)
                                y_data.insert(ss, y_data[-1])

                                tran_img = img.rotate(0, translate = k)
                                np_img = np.array(tran_img)
                                x_data.insert(ss, np_img)  



                except FileNotFoundError:
                    err +=1
                    print(json_path)

        print('ABS:', abb)
        print('Normal', norm_num)
        print('err:', err)



        num_drop = 2*len(norm_idx) -len(x_data)  #len(norm_idx) - len(x_data)-(len(norm_idx) )

        remo = sorted(random.sample(norm_idx,num_drop))

        for i in range(len(remo)):
            del x_data[remo[-i-1]]
            del y_data[remo[-i-1]]
        
        x_data = np.array(x_data)
        y_data = np.array(y_data)


        if train_val == 't':

            a_path = "C:/Users/seonh/Downloads/062.반려동물 질병 진단을 위한 영상 데이터 (근골격계)/01.데이터/Training/augment"

            np.save(a_path +'/'+di+ pos_spec +'_x_data_tr_ch_1000.npy', x_data)
            np.save(a_path +'/'+di+  pos_spec +'_y_data_tr_ch_1000.npy', y_data)

        else:

            dir_path = "C:/Users/seonh/Downloads/062.반려동물 질병 진단을 위한 영상 데이터 (근골격계)/01.데이터/Validation/numpy_save"

            np.save(dir_path +'/'+di+pos_spec+'_x_data_val_1000.npy', x_data)
            np.save(dir_path +'/'+di+pos_spec+'_y_data_val_1000.npy', y_data)

  1%|▏         | 1558/113725 [00:06<07:40, 243.44it/s]


KeyboardInterrupt: 

In [ ]:
err = 0
norm_num = 0
abb = 0
numb = -1
x_data, y_data = [], []
resize = 224
norm_idx = []

rot = [-5, 5]
trans = [[resize/10, 0], [0,(resize/10)] ] #, [0,-(resize/10)], [-(resize/10), 0]]

V, L, A, M = [], [], [], []
ab_v, ab_l, ab_a, ab_m = 0, 0, 0, 0

pos_spec = ['_A']

for json_name  in tqdm(json_name_list):
    if json_name.split('_')[6] == di:
        json_path = json_dir_path + '/'+ json_name
        try:
            with open(json_path, 'r', encoding='utf-8') as f:
                
                
                data = json.load(f)
                status = data["metadata"]['Disease']
                pose = data["metadata"]['Position']
                int_status = status_to_int(status)
                
                
                if pose == "VD":
                    continue
                    numb +=1
                    V.append(numb)
                    
                    
                    
                elif pose == "Lateral":
                    #continue
                    numb +=1
                    L.append(numb)
                    
                    
                    
                elif pose == "AP":
                    continue
                    numb +=1
                    A.append(numb)
                    
                    
                    
                    
                else:
                    continue
                    numb +=1
                    
                    M.append(numb)
                    #continue
                    
                    
                    
                  
                #y_data.append(numb)
                y_data.append(int_status)
            
                    
                img_path = json_name_to_img_path(json_name)
                
                
                annotation = data["annotations"][0]['points']
                [[x,y], [w,h]] =  annotation
                
                # 결측치 조절
                if x> w:
                    x, w = w,x
                    
                if y>h:
                    y, h = h,y
                    
                    
                box_x = w-x
                box_y = h-y

                img_path = json_name_to_img_path(json_name)
                img = Image.open(img_path).convert('1')
                
                xx = img.size[0]/10
                yy = img.size[1]/10
                
                if xx>x or yy> y or w+xx>img.size[0] or h+yy>img.size[1]:
    
                    cropped_img = img.crop((x-xx, y-yy, w+xx, h+yy))      
                    img = cropped_img

                    
                    
                                
            img = padding(img_path)    
            img_f = np.array(img)
            

            
            x_data.append(img_f)


            
            if y_data[-1] == 0  :
                norm_num +=1
                
                norm_idx.append(numb)
                if numb >=len(x_data):
                    print(numb, 'x 길이', len(x_data))
                    
                if train_val == 't':
                    for angle in rot:   #test_img = test_img.rotate(-5)
                        ss = randrange(len(x_data)+1)
                        y_data.insert(ss, y_data[-1])

                        rot_img = img.rotate(angle)
                        np_img = np.array(rot_img)
                        x_data.insert(ss, np_img)

                        norm_idx.append(ss)

                    for k in trans:
                        ss = randrange(len(x_data)+1)
                        y_data.insert(ss, y_data[-1])

                        tran_img = img.rotate(0, translate = k)
                        np_img = np.array(tran_img)
                        x_data.insert(ss, np_img)

                        norm_idx.append(ss)

                
            else:
                abb +=1
                
                if pose == "VD":
                    ab_v+= 1
                    
                elif pose == "Lateral":
                    ab_l +=1
                elif pose == "AP":
                    ab_a +=1
                else:
                    ab_m +=1
                
                if train_val == 't':
                    for angle in rot:   #test_img = test_img.rotate(-5)
                        ss = randrange(len(x_data)+1)
                        y_data.insert(ss, y_data[-1])

                        rot_img = img.rotate(angle)
                        np_img = np.array(rot_img)
                        x_data.insert(ss, np_img)
                        #train_set.append(value)

                    for k in trans:
                        ss = randrange(len(x_data)+1)
                        y_data.insert(ss, y_data[-1])

                        tran_img = img.rotate(0, translate = k)
                        np_img = np.array(tran_img)
                        x_data.insert(ss, np_img)  

                
                
        except FileNotFoundError:
            err +=1
            print(json_path)

print('ABS:', abb)
print('Normal', norm_num)
print('err:', err)

  9%|▉         | 10236/113725 [01:27<40:06, 43.01it/s]  